In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
import random
from new_Arm_Lib import Arm_Device


Arm = Arm_Device()
time.sleep(.1)

# convert speed from degree/second to millisecond
def speed_conversion(speed, start_angle, desired_angle):
    step = abs(desired_angle - start_angle)
    time = step/speed
    milli_time = round(time * 1000)
    time_sec = round(time)desired_angle
    return milli_time, time_sec

def main():
    Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 500)
    time.sleep(1)
    
    #desired_angle = int(input())
    desired_angle = 70
    
    qmeasured = [0.0]*6
    error = [0.0]*6
    error_prev = [0.0]*6
    error_sum = [0.0]*6
    pid_speed = [0.0]*6
    pid_speed_angle = [0.0]*6

    # PID values: 
    KP = 0.2
    KD = 0.1
    KI = 0.05
    
    
    while(True):
        
        for i in range(0,6): 
            # calculate the angle error
            error[i] = abs(float(desired_angle) - float(qmeasured[i]))
            
            # calculate pid speed for each servo
            speed_i = error[i] * KP + error_prev[i] * KD + error_sum[i] * KI
            
            # read the current arm angle
            start_angle = Arm.Arm_serial_servo_read(i+1)
            time.sleep(0.5)
            qmeasured[i] = start_angle
            
            # store the speed to pid_speed_angle
            pid_speed_angle[i] = speed_i
            
            # convert the speed from degree/second to millisecond
            # and store it into pid_speed list
            pid_speed[i], time_sec_i = speed_conversion(speed_i, start_angle, desired_angle)
            
            # move the servo by using the calculated time
            print("Moving servo %d, speed: %d degree/sec" %(i, speed_i))
            Arm.Arm_serial_servo_write(i+1, desired_angle, pid_speed[i])
            time.sleep(max(time_sec_i,0.5))
            
            # calculate the error
            error_prev[i] = abs(error[i])
            error_sum[i] = abs(error[i] + error_sum[i])
            
        time.sleep(1)
        print("Speed: ", pid_speed)
        print("error: ", error)
        print("error_prev: ", error_prev)
        print("error_sum: ", error_sum)


